In [1]:
# 加载包
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
import csv
import time
import pandas as pd
import datetime
import bs4

In [2]:
# 参数设置
## 增加重连次数
requests.adapters.DEFAULT_RETRIES = 5
base_url = 'http://www.ochirly.com.cn'
url_mid = 'list-'
url_end = '-40-listedDate%20desc-0-0-0-0-0.shtml'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 UBrowser/6.1.2107.204 Safari/537.36', 
          'Connection':'close'}

In [4]:
# 读取商品分类信息:ochirly_category.csv
ochirly_category = pd.read_table('./ochirly_category.csv', 
                                 sep = ',', 
                                 encoding = 'gbk')
ochirly_category

,cat1,cat2,url,page_num
0,新品,3月到店,/New_In/New_in_March/,5
1,新品,4月3日到店,/New_In/New_in_4_3/,4
2,新品,4月10日到店,/New_In/New_in_4_10/,1
3,新品,4月17日到店,/New_In/New_in_4_17/,5
4,新品,4月24日到店,/New_In/New_in_4_24/,1
5,服装,连衣裙,/Clothing/Dresses/,11
6,服装,毛织,/Clothing/Sweaters/,18
7,服装,T恤,/Clothing/T_Shirts/,12
8,服装,衬衫,/Clothing/Shirts/,9
9,服装,半截裙,/Clothing/Skirts/,10


In [5]:
# 爬取新品的上市日期

print('开始爬取新品的上市日期')
xp_start_time = datetime.datetime.fromtimestamp(time.time())
xp_start_time2 = datetime.datetime.strftime(xp_start_time, '%H:%M:%S')
print('开始时间： ' + xp_start_time2)

## 从ochirly_category表中筛选新品
ochirly_newin = ochirly_category.loc[ochirly_category.cat1 == '新品',]
## 重置row index为从0开始
ochirly_newin = ochirly_newin.reset_index(drop = True)
## 打开一个 csv 文件，把goods_id 和上市日期写入
csv_file = open('./ochirly_goods_newin.csv', 
                'w', 
                newline = '', 
                encoding = 'utf-8-sig')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['goods_id', 'newin_date'])
## 获取 goods_id 和上市日期
for row_num in range(ochirly_newin.shape[0]):
    newin_date = ochirly_newin.loc[row_num, 'cat2'].replace('到店', '').replace('月', '-').replace('日','')
    for page_num in range(ochirly_newin.loc[row_num, 'page_num']):
        url1 = base_url + ochirly_newin.loc[row_num, 'url'] + url_mid + str(page_num+1) + url_end
        page1 = requests.get(url1, headers = header)
        source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
        goods_list = source1.find('div', class_ = 'product_list')
        ## 为避免出现NoneType对象没有属性find_all，用isinstance()函数将空类型过滤掉
        if isinstance(goods_list, bs4.element.Tag):
            goods_infos = goods_list.find_all('li')
            for goods_info in goods_infos:
                goods_href = goods_info.find('a').get('href')
                url2 = base_url + goods_href
                page2 = requests.get(url2, headers = header)
                source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
                goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
                ## 把结果写入到 csv 文件中
                csv_writer.writerow([goods_id, newin_date])
            # 休息一下
            time.sleep(3)
# 关闭 csv_file
csv_file.close()

print('新品上市日期爬取完成！')
xp_end_time = datetime.datetime.fromtimestamp(time.time())
xp_end_time2 = datetime.datetime.strftime(xp_end_time, '%H:%M:%S')
print('完成时间： ' + xp_end_time2)
xp_use_time = xp_end_time - xp_start_time
print('用时： ' + str(xp_use_time.seconds) + ' 秒')
print('=========================')

开始爬取新品的上市日期
开始时间： 16:33:58
新品上市日期爬取完成！
完成时间： 16:36:35
用时： 157 秒


In [6]:
# ==================================================================
# 爬取所有服装商品
# 多个大类一起爬取
# ==================================================================

print('开始爬取服装')
fz_start_time = datetime.datetime.fromtimestamp(time.time())
fz_start_time2 = datetime.datetime.strftime(fz_start_time, '%H:%M:%S')
print('开始时间： ' + fz_start_time2)

## 从ochirly_category表中筛选服装的信息
ochirly_yifu = ochirly_category.loc[ochirly_category.cat1 == '服装',]
## 重置row index为从0开始
ochirly_yifu = ochirly_yifu.reset_index(drop = True)
## 打开一个 csv 文件，写入商品信息
csv_file = open('./ochirly_goods_info.csv', 
                'w', 
                newline = '', 
                encoding = 'utf-8-sig')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['category','goods_id','goods_name','color','sale_price','origin_price',
                     'design', 'material', 'stereotype', 'length', 'page_num'])
## 获取商品信息
for row_num in range(ochirly_yifu.shape[0]):
    category = ochirly_yifu.loc[row_num, 'cat2']
    for page_num in range(ochirly_yifu.loc[row_num, 'page_num']):
        url1 = base_url + ochirly_yifu.loc[row_num, 'url'] + url_mid + str(page_num+1) + url_end
        page1 = requests.get(url1, headers = header)
        source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
        goods_infos = source1.find('div', class_ = 'product_list').find_all('li')
        for goods_info in goods_infos:
            goods_href = goods_info.find('a').get('href')
            url2 = base_url + goods_href
            page2 = requests.get(url2, headers = header)
            source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
            goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
            try:
                goods_name = source2.find('p', class_ = 'title').string.replace('\n', '').replace('	', '')
            except Exception as e:
                goods_name = None
            try:
                color = source2.find('div', class_ = 'color clearfix').a.find('div', class_ = 'arrow').p.string
            except Exception as e:
                color = None
            try:
                price = source2.find('p', class_ = 'price').em.children
                sale_price = next(price).replace('￥','')
                origin_price = next(price).string.replace('￥','')
            except Exception as e:
                sale_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
                origin_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
            try:
                design = source2.find(attrs = {'id':'detailContent1'}).find_all('li')[0].p.string
            except Exception as e:
                design = None
            try:
                material = source2.find(attrs = {'id':'detailContent1'}).find_all('li')[1].p.string.replace('面料:','')
            except Exception as e:
                material = None
            try:
                stereotype = source2.find(attrs = {'id':'detailContent2'}).find_all('li')[0].p.string
            except Exception as e:
                stereotype = None
            try:
                length = source2.find(attrs = {'id':'detailContent2'}).find_all('li')[1].p.string
            except Exception as e:
                length = None
            ## 把结果写入到 csv 文件中
            csv_writer.writerow([category,goods_id,goods_name,color,sale_price,origin_price,
                                 design,material,stereotype,length,page_num+1])
            
            ## 由于图片比较耗资源，暂时不爬取图片
            '''
            ## 爬取主图片
            main_src = source2.find('ul', class_ = 'lineone').find('div', class_ = 'big_img').find('img').get('data-src')
            img = requests.get(main_src, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            ## 爬取展示图片
            show_src1 = source2.find('ul', class_ = 'linetwo clearfix').find_all('li')[0].find('img').get('data-src')
            img = requests.get(show_src1, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-show1' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            show_src2 = source2.find('ul', class_ = 'linetwo clearfix').find_all('li')[1].find('img').get('data-src')
            img = requests.get(show_src2, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-show2' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            show_src3 = source2.find('ul', class_ = 'linetwo clearfix').find_all('li')[2].find('img').get('data-src')
            img = requests.get(show_src3, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-show3' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            show_src4 = source2.find('ul', class_ = 'linetwo clearfix').find_all('li')[3].find('img').get('data-src')
            img = requests.get(show_src4, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-show4' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            ## 爬取细节图片
            detail_src1 = source2.find('ul', class_ = 'linethree clearfix').find_all('li')[0].find('img').get('data-src')
            img = requests.get(detail_src1, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-detail1' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            detail_src2 = source2.find('ul', class_ = 'linethree clearfix').find_all('li')[1].find('img').get('data-src')
            img = requests.get(detail_src2, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-detail2' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            detail_src3 = source2.find('ul', class_ = 'linethree clearfix').find_all('li')[2].find('img').get('data-src')
            img = requests.get(detail_src3, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-detail3' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            '''
        # 显示进度
        print(category, '第', page_num+1, '页')
        print('------------')
        # 每爬取一页，休息一下
        time.sleep(30)
# 每爬取一个品类，休息一下
time.sleep(300)
# 关闭 csv_file
csv_file.close()

print('服装爬取完成！')
fz_end_time = datetime.datetime.fromtimestamp(time.time())
fz_end_time2 = datetime.datetime.strftime(fz_end_time, '%H:%M:%S')
print('完成时间： ' + fz_end_time2)
fz_use_time = fz_end_time - fz_start_time
print('用时： ' + str(fz_use_time.seconds) + ' 秒')
print('=========================')


开始爬取服装
开始时间： 16:43:10
连衣裙 第 1 页
------------
连衣裙 第 2 页
------------
连衣裙 第 3 页
------------
连衣裙 第 4 页
------------
连衣裙 第 5 页
------------
连衣裙 第 6 页
------------
连衣裙 第 7 页
------------
连衣裙 第 8 页
------------
连衣裙 第 9 页
------------
连衣裙 第 10 页
------------
连衣裙 第 11 页
------------
毛织 第 1 页
------------
毛织 第 2 页
------------
毛织 第 3 页
------------
毛织 第 4 页
------------
毛织 第 5 页
------------
毛织 第 6 页
------------
毛织 第 7 页
------------
毛织 第 8 页
------------
毛织 第 9 页
------------
毛织 第 10 页
------------
毛织 第 11 页
------------
毛织 第 12 页
------------
毛织 第 13 页
------------
毛织 第 14 页
------------
毛织 第 15 页
------------
毛织 第 16 页
------------
毛织 第 17 页
------------
毛织 第 18 页
------------
T恤 第 1 页
------------
T恤 第 2 页
------------
T恤 第 3 页
------------
T恤 第 4 页
------------
T恤 第 5 页
------------
T恤 第 6 页
------------
T恤 第 7 页
------------
T恤 第 8 页
------------
T恤 第 9 页
------------
T恤 第 10 页
------------
T恤 第 11 页
------------
T恤 第 12 页
------------
衬衫 第 1 页
------------
衬衫 第 2 页
------------
衬衫 第 3 

In [4]:
# 爬取商品的场景分类

print('开始爬取商品的场景分类')
cj_start_time = datetime.datetime.fromtimestamp(time.time())
cj_start_time2 = datetime.datetime.strftime(cj_start_time, '%H:%M:%S')
print('开始时间： ' + cj_start_time2)

## 从ochirly_category表中筛选场景购的信息
ochirly_changjing = ochirly_category.loc[ochirly_category.cat1 == '场景',]
## 重置row index为从0开始
ochirly_changjing = ochirly_changjing.reset_index(drop = True)
## 打开一个 csv 文件，把goods_id 和场景名称写入
csv_file = open('./ochirly_goods_changjing.csv', 
                'w', 
                newline = '', 
                encoding = 'utf-8-sig')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['goods_id', 'changjing'])
## 获取 goods_id 和场景名称
for row_num in range(ochirly_changjing.shape[0]):
    changjing = ochirly_changjing.loc[row_num, 'cat2']
    for page_num in range(ochirly_changjing.loc[row_num, 'page_num']):
        url1 = base_url + ochirly_changjing.loc[row_num, 'url'] + url_mid + str(page_num+1) + url_end
        page1 = requests.get(url1, headers = header)
        source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
        goods_infos = source1.find('div', class_ = 'product_list').find_all('li')
        for goods_info in goods_infos:
            goods_href = goods_info.find('a').get('href')
            url2 = base_url + goods_href
            page2 = requests.get(url2, headers = header)
            source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
            goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
            ## 把结果写入到 csv 文件中
            csv_writer.writerow([goods_id, changjing])
        # 休息一下
        time.sleep(3)
# 关闭 csv_file
csv_file.close()

print('场景分类爬取完成！')
cj_end_time = datetime.datetime.fromtimestamp(time.time())
cj_end_time2 = datetime.datetime.strftime(cj_end_time, '%H:%M:%S')
print('完成时间： ' + cj_end_time2)
cj_use_time = cj_end_time - cj_start_time
print('用时： ' + str(cj_use_time.seconds) + ' 秒')
print('=========================')

开始爬取商品的场景分类
开始时间： 14:14:34
场景分类爬取完成！
完成时间： 14:15:15
用时： 40 秒


In [20]:
# ==================================================================
# 连续爬取多个品类中途容易中断，因此单独爬取一个品类 =======================
# ==================================================================

print('开始爬取')
fz_start_time = datetime.datetime.fromtimestamp(time.time())
fz_start_time2 = datetime.datetime.strftime(fz_start_time, '%H:%M:%S')
print('开始时间： ' + fz_start_time2)

## 选择需要爬取的品类
ochirly_yifu = ochirly_category.loc[ochirly_category.cat2 == '连衣裙',]
## 重置row index为从0开始
ochirly_yifu = ochirly_yifu.reset_index(drop = True)
## 打开一个 csv 文件，写入商品信息
csv_file = open('./ochirly_goods_info.csv', 
                'w', 
                newline = '', 
                encoding = 'utf-8-sig')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['category','goods_id','goods_name','color','sale_price','origin_price',
                     'design', 'material', 'stereotype', 'length', 'page_num'])
## 获取商品信息
category = ochirly_yifu.loc[0, 'cat2']
for page_num in range(ochirly_yifu.loc[0, 'page_num']):
    s1 = requests.session()
    s1.keep_alive = False
    url1 = base_url + ochirly_yifu.loc[0, 'url'] + url_mid + str(page_num+1) + url_end
    page1 = s1.get(url1, headers = header)
    source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
    goods_infos = source1.find('div', class_ = 'product_list').find_all('li')
    for goods_info in goods_infos:
        goods_href = goods_info.find('a').get('href')
        s2 = requests.session()
        s2.keep_alive = False
        url2 = base_url + goods_href
        page2 = s2.get(url2, headers = header)
        source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
        goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
        try:
            goods_name = source2.find('p', class_ = 'title').string.replace('\n', '').replace('	', '')
        except Exception as e:
            goods_name = None
        try:
            color = source2.find('div', class_ = 'color clearfix').find('a', class_ = 'item active').find('div', class_ = 'arrow').p.string
        except Exception as e:
            color = None
        try:
            price = source2.find('p', class_ = 'price').em.children
            sale_price = next(price).replace('￥','')
            origin_price = next(price).string.replace('￥','')
        except Exception as e:
            sale_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
            origin_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
        try:
            design = source2.find(attrs = {'id':'detailContent1'}).find_all('li')[0].p.string
        except Exception as e:
            design = None
        try:
            material = source2.find(attrs = {'id':'detailContent1'}).find_all('li')[1].p.string.replace('面料:','')
        except Exception as e:
            material = None
        try:
            stereotype = source2.find(attrs = {'id':'detailContent2'}).find_all('li')[0].p.string
        except Exception as e:
            stereotype = None
        try:
            length = source2.find(attrs = {'id':'detailContent2'}).find_all('li')[1].p.string
        except Exception as e:
            length = None
        ## 把结果写入到 csv 文件中
        csv_writer.writerow([category,goods_id,goods_name,color,sale_price,origin_price,
                             design,material,stereotype,length,page_num+1])

    # 显示进度
    print(category, '第', page_num+1, '页')
    print('------------')
    # 休息一下
    time.sleep(10)
# 关闭 csv_file
csv_file.close()

print('爬取完成！')
fz_end_time = datetime.datetime.fromtimestamp(time.time())
fz_end_time2 = datetime.datetime.strftime(fz_end_time, '%H:%M:%S')
print('完成时间： ' + fz_end_time2)
fz_use_time = fz_end_time - fz_start_time
print('用时： ' + str(fz_use_time.seconds) + ' 秒')
print('=========================')

开始爬取
开始时间： 11:44:45
连衣裙 第 1 页
------------
连衣裙 第 2 页
------------
连衣裙 第 3 页
------------
爬取完成！
完成时间： 11:46:10
用时： 84 秒


In [42]:
## 关于商品的售价和吊牌价

url2 = 'http://www.ochirly.com.cn/p/1GZ3089890090.shtml'
page2 = requests.get(url2, headers = header)
source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
price = source2.find('p', class_ = 'price')
#sale_price = next(price).replace('￥','')
#origin_price = next(price).string.replace('￥','')

'''
try:
    price = source2.find('p', class_ = 'price').em.children
    sale_price = next(price).replace('￥','')
    origin_price = next(price).string.replace('￥','')
except Exception as e:
    sale_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
    origin_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')

'''

print('goods_id:', goods_id)
#print('sale_price:' + sale_price)
#print('origin_price' + origin_price)
print(price)

goods_id: 1GZ3089890090
<p class="price" data-group-id="895#" data-group-price="895:1:399.0:CNY:OCH-官网-在售折扣品(大于5折款)#" data-list-price="669" data-offer-price="669" data-sku="1GZ3089890090" id="priceOfSku1GZ3089890090">
							￥669
						</p>


In [14]:
url2 = 'http://www.ochirly.com.cn/p/1JY4330590030.shtml'
page2 = requests.get(url2, headers = header)
source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
try:
    goods_name = source2.find('p', class_ = 'title').string.replace('\n', '').replace('	', '')
except Exception as e:
    goods_name = None
try:
    color = source2.find('div', class_ = 'color clearfix').find('a', class_ = 'item active').find('div', class_ = 'arrow').p.string
except Exception as e:
    color = None
try:
    price = source2.find('p', class_ = 'price').em.children
    sale_price = next(price).replace('￥','')
    origin_price = next(price).string.replace('￥','')
except Exception as e:
    sale_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
    origin_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
try:
    design = source2.find(attrs = {'id':'detailContent1'}).find_all('li')[0].p.string
except Exception as e:
    design = None
try:
    material = source2.find(attrs = {'id':'detailContent1'}).find_all('li')[1].p.string.replace('面料:','')
except Exception as e:
    material = None
try:
    stereotype = source2.find(attrs = {'id':'detailContent2'}).find_all('li')[0].p.string
except Exception as e:
    stereotype = None
try:
    length = source2.find(attrs = {'id':'detailContent2'}).find_all('li')[1].p.string
except Exception as e:
    length = None
print('goods_id:', goods_id)
print('goods_name:', goods_name)
print('color:', color)
print('sale_price:', sale_price)
print('origin_price:', origin_price)

goods_id: 1JY4330590030
可拆帽印花羽绒服
color: 灰色
sale_price: 1090 
origin_price: 1890


In [8]:
## 单独查询一个商品
url = 'http://www.ochirly.com.cn/New_In/New_in_1_23/list.shtml'
page1 = requests.get(url, headers = header)
source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
goods_info = source1.find('div', class_ = 'product_list').find('li')
goods_href = goods_info.find('a').get('href')
url2 = base_url + goods_href
page2 = requests.get(url2, headers = header)
source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
print(goods_id)

1ZZ1018880530


In [12]:
## 查询多个商品
url1 = 'http://www.ochirly.com.cn/p/1GY4330780532.shtml'
page1 = requests.get(url1, headers = header)
source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
goods_infos = source1.find('div', class_ = 'product_list')##.find_all('li')
print(goods_infos)
print(type(goods_info))
'''
for goods_info in goods_infos:
    goods_href = goods_info.find('a').get('href')
    url2 = base_url + goods_href
    page2 = requests.get(url2, headers = header)
    source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
    goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
    print(goods_id)
'''


None
<class 'bs4.element.Tag'>


"\nfor goods_info in goods_infos:\n    goods_href = goods_info.find('a').get('href')\n    url2 = base_url + goods_href\n    page2 = requests.get(url2, headers = header)\n    source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')\n    goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')\n    print(goods_id)\n"

In [101]:
url1 = 'http://www.ochirly.com.cn/p/1GY4330780532.shtml'
page1 = requests.get(url1, headers = header)
source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
img_src = source1.find('ul', class_ = 'lineone')
print(img_src)

<ul class="lineone">
<li>
<div class="zoom_wrap" id="zoomWrap">
<div class="middle_img" id="middleImg">
<img alt="翻领长款羽绒服 " data-src="http://img1.ochirly.com.cn/wcsstore/TrendyCatalogAssetStore/images/trendy/ochirly/2018/d/1GY4330780532/1GY4330780532_m_8.jpg" src="//img2.ochirly.com.cn/rs/common/images/blank.gif"/>
</div>
<div class="big_img" id="bigImg">
<img alt="翻领长款羽绒服 " data-src="http://img1.ochirly.com.cn/wcsstore/TrendyCatalogAssetStore/images/trendy/ochirly/2018/d/1GY4330780532/1GY4330780532_b_8.jpg" src="//img2.ochirly.com.cn/rs/common/images/blank.gif"/>
</div>
</div>
</li>
</ul>


In [51]:
print(ochirly_newin.loc[7,'cat2'])

9月5日到店


In [48]:
ochirly_newin = ochirly_category.loc[ochirly_category.cat1 == '新品',]
print(ochirly_newin.shape)
## 获取 goods_id 和场景名称
for row_num in range(ochirly_newin.shape[0]):
    #newin_date = ochirly_newin.loc[row_num, 'cat2']
    print(row_num)
    #print(newin_date)

(5, 4)
0
1
2
3
4


In [35]:
url2 = 'http://www.ochirly.com.cn/p/1GH2085730510.shtml'
page2 = requests.get(url2, headers = header)
source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
print(goods_href)
print(goods_id)
print(goods_price)

/p/1GY1084160090.shtml
1GH2085730510
						


In [44]:
ochirly_changjing = ochirly_category.loc[ochirly_category.cat1 == '场景',]
## 获取 goods_id 和场景名称
for row_num in range(ochirly_changjing.shape[0]):
    changjing = ochirly_changjing.loc[row_num, 'cat2']
    print(row_num)
    print(changjing)

0
休闲
1
工作
2
逛街约会
3
度假
4
派对
5
SNOOPY限量版
